In [3]:
import numpy as np
from sklearn.cluster import KMeans
from scipy.io import wavfile
import os
from python_speech_features import mfcc
from python_speech_features import logfbank
from scipy.stats import multivariate_normal as MVN

In [4]:
def kmean(data, K=3):
      
        kmeans = KMeans(n_clusters=K, random_state=42).fit(data)
        clu = kmeans.labels_
        cov = []
        for k in range(K):
            var = []
            for i in range(data.shape[0]):
                if clu[i] == k:
                    var.append(data[i])

            var = np.array(var)
            _cov = np.cov(var.T)
            cov.append(_cov)
        return kmeans.cluster_centers_, np.array(cov) + np.random.random(np.shape(cov))
    

In [53]:
class HMM:
    def __init__(self, Data , K=3 , maxiter=30 ):
        self.K = K
        
        #Initializing Parameters
        self.pik = np.random.random(K)
        self.pik = self.pik/np.sum(self.pik)
        self.A = np.random.random([K,K])
        #self.A = np.triu(self.A, 0)
        self.A = self.A/np.sum(self.A,axis=1)
        self.maxiter = maxiter
        #Initializing mean and variance through KMEANS
        self.mu , self.sigma = kmean(Data,K)
            

    def train(self , X1):
        K = self.K
        pik = self.pik
        A = self.A
        mu = self.mu
        sigma = self.sigma
        _=0
        for _ in range(self.maxiter):
            print(_)
            X = X1[_%len(X1)]
            # Initalizing Alpha
            Alpha1 = []
            for i in range(K):
                j = pik[i]
                j*=MVN.pdf(X[0],mean=mu[i],cov=sigma[i])
                Alpha1.append(j)
                
            C1 = np.sum(Alpha1)
            Alpha1 = np.array(Alpha1)/C1
            Alpha = np.zeros([X.shape[0],K])
            C = np.zeros(X.shape[0])
            C[0] = C1
            Alpha[0,:] = np.array(Alpha1)
            
            
            #Forward Recursion to calculate all alpha (Normalized)
            for i in range(1, X.shape[0]):
                for j in range(K):
                    Sum = 0
                    for k in range(K):
                        Sum += Alpha[i-1,k]*A[k,j]
                    Sum*=MVN.pdf(X[i],mu[j],sigma[j])
                    Alpha[i,j] = Sum
                C[i] = np.sum(Alpha[i])   #Normalining Alpha
                Alpha[i]/=C[i]
            
            
            #Initialing Beta[Zn] as 1
            Beta = np.zeros([X.shape[0],K])
            Beta[-1,:] = 1
            
            #Backward Recursion to calculate all beta
            for i in range(X.shape[0]-2 ,-1,-1 ):
                for j in range(K):
                    Sum=0
                    for k in range(K):
                        Sum+=Beta[i+1,k]*MVN.pdf(X[i+1],mu[k],sigma[k])*A[j,k]
                    Beta[i,j]=Sum/C[i+1]
              
            
            # Updating Parameters
            neta = Alpha*Beta
            
            epsilon = np.zeros([X.shape[0],K,K])
            for i in range(1,X.shape[0]):
                for j in range(K):
                    for k in range(K):
                        epsilon[i,j,k] = Alpha[i-1,j]*Beta[i,k]*MVN.pdf(X[i],mu[k],sigma[k])*A[j,k]*C[i]
                        
                        
            # Updating PIk (Probabilites of Z1) 
            pik = (neta[0])/np.sum(neta)
            
            #Updating Transition probability matrix A (P(Zn/Zn-1))
            
            for j in range(self.K):
                for k in range(self.K):
                    A[j][k] = np.sum(epsilon[1:][j][k])/(np.sum(epsilon[1:][j][:]))

            
            # Updating mean of Multivariate normal (Mu_k)
            for j in range(K):
                mu[j] =0
                for i in range(X.shape[0]):
                    mu[j] += neta[i,j]*X[i]
                mu[j] = mu[j]/(np.sum(neta[:,j]))
    
    
            # Updating Covariance of Multivariate normal (Sigma_k)
            for j in range(K):
                sigma[j] =0
                for i in range(X.shape[0]):
                    temp = X[i]-mu[j]
                    temp = temp.reshape(1,-1)
                    sigma[j] += neta[i,j]*np.dot(temp,temp.T)
                sigma[j] = sigma[j]/(np.sum(neta[:,j]))
                
            vec=np.random.random(13)*50
            mat=np.diag(vec)
            
            sigma+=mat
                
            
            err1 = np.sum(np.abs(A-self.A))
            err2 = np.sum(np.abs(pik-self.pik))
            err3 = np.sum(np.abs(mu-self.mu))
            err4 = np.sum(np.abs(sigma-self.sigma))
            
            self.pik = pik
            self.A = A
            self.mu = mu
            self.sigma = sigma
            
            
            #Calculating Log Liklihood
            S1 = 0
            for i in range(1, X.shape[0]):
                for k in range(K):
                    for j in range(K):
                        S1+=epsilon[i,j,k]*np.log(A[j,k])
            
            S2 = 0
            for i in range(X.shape[0]):
                 for j in range(K):
                     S2+=neta[i,k]*np.log(MVN.pdf(X[i] , mu[k] , sigma[k]))

            liklihood = np.sum(neta[0]*np.log(pik)) + S1 + S2
            
            print("At " , _ , "  Liklihood :" , liklihood)
            
            
            #Checking Stopping condition
            #if(err1+err2+err3+err4 < 1e-103 ):
                #break
            
            
    def test(self,X):
            #Testing Function calculates Log Liklihood for a given sample.

            K = self.K
            pik = self.pik
            A = self.A
            mu = self.mu
            sigma = self.sigma
            Alpha1 = []
            for i in range(K):
                j = pik[i]
                j*=MVN.pdf(X[0],mean=mu[i],cov=sigma[i])
                Alpha1.append(j)
                
            
            
            C1 = np.sum(Alpha1)
            Alpha1 = np.array(Alpha1)/C1
            Alpha = np.zeros([X.shape[0],K])
            C = np.zeros(X.shape[0])
            C[0] = C1
            Alpha[0,:] = np.array(Alpha1)
            
            
            #Calculating Normalized Alpha
            for i in range(1, X.shape[0]):
                for j in range(K):
                    Sum = 0
                    for k in range(K):
                        Sum += Alpha[i-1,k]*A[k,j]
                    Sum*=MVN.pdf(X[i],mu[j],sigma[j])
                    Alpha[i,j] = Sum
                C[i] = np.sum(Alpha[i])
                Alpha[i]/=C[i]
            
            
            #Calculating Normalized Beta
            Beta = np.zeros([X.shape[0],K])
            Beta[-1,:] = 1
            for i in range(X.shape[0]-2 ,-1,-1 ):
                for j in range(K):
                    Sum=0
                    for k in range(K):
                        Sum+=Beta[i+1,k]*MVN.pdf(X[i+1],mu[k],sigma[k])*A[j,k]
                    Beta[i,j]=Sum/C[i+1]
                    
            neta = Alpha*Beta
            epsilon = np.zeros([X.shape[0],K,K])
            for i in range(1,X.shape[0]):
                for j in range(K):
                    for k in range(K):
                        epsilon[i,j,k] = Alpha[i-1,j]*Beta[i,k]*MVN.pdf(X[i],mu[k],sigma[k])*A[j,k]*C[i]
                        
            
            #Calculating Log Liklihood
#             S1 = 0
#             for i in range(1, X.shape[0]):
#                 for k in range(K):
#                     for j in range(K):
#                         S1+=epsilon[i,j,k]*np.log(A[j,k])
            
#             S2 = 0
#             for i in range(X.shape[0]):
#                 for j in range(K):
#                     S2+=neta[i,k]*np.log(MVN.pdf(X[i] , mu[k] , sigma[k]))

            #liklihood = np.sum(neta[0]*np.log(pik)) + S1 + S2
            liklihood = np.log(np.sum(Alpha[-1]))
            return liklihood
            
            
        

In [54]:
path = "/home/aayush/Downloads/syllable/SVL-PRASANNA-CV/tho/"
L = os.listdir(path)
rate, sig = wavfile.read(path+L[0])

#Extracting MFCC features
X = mfcc(sig,rate)
Data = X[:]
TT = [X]

for i in L[1:]:
    rate, sig = wavfile.read(path+i)  
    _Temp = mfcc(sig,rate)
    TT.append(_Temp)
    Data = np.append(Data, _Temp, axis=0)
    
#Extracting last 2 speech samples for testing
TestData = [TT.pop()]+ [TT.pop()]



In [55]:
hmm = HMM(Data,1)
hmm.train(TT)

0
At  0   Liklihood : -3523.5580074281584
1
At  1   Liklihood : -2849.719537211942
2
At  2   Liklihood : -6932.250144425958
3
At  3   Liklihood : -3995.4782552262536
4
At  4   Liklihood : -2991.76174801499
5
At  5   Liklihood : -5788.219429494324
6
At  6   Liklihood : -3354.977396044084
7
At  7   Liklihood : -2822.654155115572
8
At  8   Liklihood : -3029.838506103283
9
At  9   Liklihood : -3816.7354972404573
10
At  10   Liklihood : -5087.814888701836
11
At  11   Liklihood : -3199.566751124943
12
At  12   Liklihood : -4539.521633393251
13
At  13   Liklihood : -3537.7960356139124
14
At  14   Liklihood : -7240.96499251596
15
At  15   Liklihood : -2930.315522545259
16
At  16   Liklihood : -7362.367744302703
17
At  17   Liklihood : -5361.232732336645
18
At  18   Liklihood : -3795.18274567633
19
At  19   Liklihood : -3042.3883821598465
20
At  20   Liklihood : -4671.050706465693
21
At  21   Liklihood : -2886.570096063266
22
At  22   Liklihood : -3461.8964056681207
23
At  23   Liklihood : -476

<ipython-input-53-870c6785f2dd>:50: RuntimeWarning: invalid value encountered in true_divide
  Alpha[i]/=C[i]


ValueError: array must not contain infs or NaNs

In [47]:
print(hmm.test(TestData[0]))
print(hmm.test(TestData[1]))

-5128.752793296907
-4136.900937112739


In [48]:
path = "/home/aayush/Downloads/syllable/SVL-PRASANNA-CV/bha/"
L = os.listdir(path)
rate, sig = wavfile.read(path+L[0])        
X = mfcc(sig,rate)
Data = X[:]
TT = [X]
for i in L[1:]:
    rate, sig = wavfile.read(path+i)  
    _Temp = mfcc(sig,rate)
    TT.append(_Temp)
    Data = np.append(Data, _Temp, axis=0)
TestData = [TT.pop()]+ [TT.pop()]

In [49]:
hmm = HMM(Data,1)
hmm.train(TT)

0
At  0   Liklihood : -6434.248743134542
1
At  1   Liklihood : -3315.0082995034186
2
At  2   Liklihood : -2674.3832832112416
3
At  3   Liklihood : -3384.405739705172
4
At  4   Liklihood : -2905.3934548596667
5
At  5   Liklihood : -4324.294502624398
6
At  6   Liklihood : -4578.378413733037
7
At  7   Liklihood : -5016.905412048205
8
At  8   Liklihood : -3923.1353911371943
9
At  9   Liklihood : -3868.1256204409674
10
At  10   Liklihood : -4629.515662339527
11
At  11   Liklihood : -3066.542908593647
12
At  12   Liklihood : -6774.259029509826
13
At  13   Liklihood : -5259.139293659609
14
At  14   Liklihood : -4014.5150742435862
15
At  15   Liklihood : -3088.7384554375963
16
At  16   Liklihood : -5855.467212338166
17
At  17   Liklihood : -4869.386133467038
18
At  18   Liklihood : -3131.2852060443306
19
At  19   Liklihood : -4385.765295775518
20
At  20   Liklihood : -3367.48586591407
21
At  21   Liklihood : -7374.265381589868
22
At  22   Liklihood : -3026.5480520474894
23
At  23   Liklihood :

<ipython-input-44-dcedf13e5492>:50: RuntimeWarning: invalid value encountered in true_divide
  Alpha[i]/=C[i]


ValueError: array must not contain infs or NaNs

In [43]:
print(hmm.test(TestData[0]))
print(hmm.test(TestData[1]))

-7479.215306085398
-4263.51233274257
